<a href="https://colab.research.google.com/github/thefr33radical/codeblue/blob/master/bin_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
/**
You are given a binary tree as a sequence of parent-child pairs. 
For example, the tree represented by the node pairs below:
(A,B) (A,C) (B,G) (C,H) (E,F) (B,D) (C,E)
may be illustrated in many ways, with two possible representations below:
     A   /  \  B    C / \  / \G  D  E   H       \            F        
     A   /  \  B    C / \  / \D  G H   E        /       F
Below is the recursive definition for the S-expression of a tree:
S-exp(node) = ( node->val 
(S-exp(node->first_child))(S-exp(node->second_child))), if node != NULL
                         = "", node == NULL 
   where, first_child->val < second_child->val (lexicographically smaller)
This tree can be represented in a S-expression in multiple ways. 
The lexicographically smallest way of expressing this is as follows:
(A(B(D)(G))(C(E(F))(H)))
We need to translate the node-pair representation into an S-expression 
(lexicographically smallest one), and report any errors that do not conform to the definition of a binary tree.
The list of errors with their codes is as follows:
Error Code          Type of error
E1                 More than 2 children
E2                 Duplicate Edges
E3                 Cycle present
E4                 Multiple roots
E5                 Any other error   
Input Format:
Input must be read from standard input.
Input will consist of on line of parent-child pairs. Each pair consists of two node names separated by a single comma and enclosed in parentheses. A single space separates the pairs.
Output:
The function must write to standard output.
Output the given tree in the S-expression representation described above.
There should be no spaces in the output.
Constraints:
There is no specific sequence in which the input (parent,child) pairs are represented.
The name space is composed of upper case single letter (A-Z) so the maximum size is 26 nodes.
Error cases are to be tagged in the order they appear on the list. For example, 
if one input pair raises both error cases 1 and 2, the output must be E1. 
Sample Input #00
(B,D) (D,E) (A,B) (C,F) (E,G) (A,C)
Sample Output #00
(A(B(D(E(G))))(C(F)))
Sample Input #01
(A,B) (A,C) (B,D) (D,C)
Sample Output #01
E3
Explanation
Node D is both a child of B and a parent of C, but C and B are both child nodes of A. 
Since D tries to attach itself as parent to a node already above it in the tree, this forms a cycle.
Idea:
Use a 26*26 graph to represent the input tree. Then check for each error in the order, finally use
a recursive DFS to form the tree and the output SExpression.
E1: More than 2 children. Check if graph[i][j], j from 0 to 25 has more than two cell that is true.
E2: Duplicate Edge. Check when constructing the graph, if graph[x][y] is already true, E2=true.
E3: Cycle Present: Check when looking for the root, starting from each root, use recursive DFS to check if there is
a cycle in the graph.
E4: Multiple roots: traverse all nodes in the HashSet, if no edge connected TO this node, then it must be a root.
即这个顶点没有入度，没有其他的点指向这个点，那么这个点必然是root. If number of roots > 1, return "E4".
Note: if number of roots == 0, then there must also be a cycle, return "E3".
*/

In [1]:


class graph(object):
  def __init__(self):
    self.v={}
    self.edge_counter={}
    self.parent={}
    
  def insert_edge(self,vertex,edge):    
    # Insert Edge into Vertex list and check for Deuplicate Edge Simultaneously
    try:
      if edge in self.v[vertex]:
        print("Error Code 2 : Duplicate Edge")
      self.v[vertex].append(edge)
    except:
      self.v[vertex]=[]
      self.v[vertex].append(edge)
      
    # Insert Edge into edget and check for more than 2 kids Simultaneously
    try:
      self.edge_counter[vertex]+=1
      if self.edge_counter[vertex]>2:
        print("Error Code 1 : More than 2 kids")
    except:
      self.edge_counter[vertex]=1  

    # Insert into parents and check for multiple parents Simultaneuosly 
    try:
      if self.parent[edge]:
        print("Error Code 3 : Multiple Parents")
    except:
        self.parent[edge]=vertex
    
# Use bfs to check for cycle
def check_cycle(graph):
  
  visited={}
  q=[]
  
  start_node = list(graph.keys())[0]
  q.append(start_node)
  visited[start_node]=1
  
  while q:
    lkid= None
    rkid=None    
    present_node = q.pop(len(q)-1)
        
    try:      
      lkid = graph[present_node][0]
    except:
      lkid = None
      
    try:      
      rkid = graph[present_node][1]
    except:
      rkid = None
      
    if lkid is not None:
        try:
          if visited[lkid]:
            return 0
        except:
          visited[lkid]=1
          q.append(lkid)
          
    if rkid is not None:
        try:
          if visited[rkid]:
            return 0
        except:
          visited[rkid]=1
          q.append(rkid)
          
  return 1

# Use DFS to convert the expresssion, post order traversal
def dfs(node,graph,expr):
  
  if node in graph:  
      expr+=node
      print("node present",node,expr)
      lkid= None
      rkid=None
      
      try:      
        lkid = graph[node][0]
        expr+="("
        expr = dfs(lkid,graph,expr)
        expr+=")"
      except:
        lkid = None
      
      try:      
        rkid = graph[node][1]
        expr+="("
        expr = dfs(rkid,graph,expr)
        expr+=")"
      except:
        rkid = None
           
      return expr       
  expr+=node
  return expr

# Convert the valid tree to Expression
def convert_exp(graph):
  start_node = "A"
  expr=""
  expr=dfs(start_node,graph,expr) 
  return expr
  
  
  
#data = input()

"""
(A,B) (A,C) (B,G) (C,H) (E,F) (B,D) (C,E)

(B,D) (D,E) (A,B) (C,F) (E,G) (A,C)
Sample Output #00
(A(B(D(E(G))))(C(F)))


Sample Input #01
(A,B) (A,C) (B,D) (D,C)
Sample Output #01
"""

data = "(A,B) (A,C) (B,D) (D,C)"
edge_list = data.split(" ")
obj=graph()
for node in edge_list:  
  obj.insert_edge(node[1],node[3])

if len(list(obj.v.keys() ))<1:
  print("Empty Graph")
  
ans = check_cycle(obj.v) 

if ans :
  # Only Acyclic graphs allowed
  print(obj.v)
  print(convert_exp(obj.v))

    #(A  (B(D)(G)) (C (E(F) ) (H) ))

Error Code 3 : Multiple Parents
